### Load Envs 

In [1]:
import os
from reasoning_engines.langchain_llm_wrappers import QuickAzureChatOpenAI, QuickAzureOpenAIClient
from tools.get_tools import DefaultSchemasTools
from langchain import hub
from utils.load_envs import Environ
from chains.qa_evaluators import CustomQAEvaluator


### ReAct

In [2]:
import os
from reasoning_engines.langchain_llm_wrappers import QuickAzureChatOpenAI, QuickAzureOpenAIClient
from tools.get_tools import DefaultSchemasTools
from langchain import hub
from utils.load_envs import Environ
from chains.qa_evaluators import CustomQAEvaluator


#### ReAct Agent
from agents.react_agents import ReActAgent

GPT_VER = '0613'
react_agent = ReActAgent(reasoninig_engine=QuickAzureChatOpenAI(version=GPT_VER), 
                        base_prompt="hwchase17/react-multi-input-json", 
                        schemas_and_tools=DefaultSchemasTools(azure_gpt_version=GPT_VER), 
                        evaluator=CustomQAEvaluator(llm=QuickAzureChatOpenAI(version='1106')), 
                        stop_words=["Observation"], 
                        thought_word= 'Thought', 
                        action_word='Action',
                        file_logging=True,
                        horizon=6)


react_agent.clear_logs()
react_agent.run_agent_trials(num_trials=2, 
query="Who is older? The former president of Korea vs. the MLB American League MVP of the last season. What is the result if you sum up the ages of the two?",
reference="The former former president of Korea is older. The sum of their age is 99.")



[INFO] Query: Who is older? The former president of Korea vs. the MLB American League MVP of the last season. What is the result if you sum up the ages of the two?
[INFO] Trial 1
[INFO] Thought 1: I need to find out who the former president of Korea is and who the MLB American League MVP of the last season is. Then, I need to find their ages. I'll start by searching for the former president of Korea.
[INFO] Action 1: YDCSearch(query='former president of Korea')
[INFO] Observation 1: The most recent former president of South Korea is Moon Jae-In, who served from 2017 to 2022.
[INFO] Thought 2: Now I need to find out the age of Moon Jae-In.
[INFO] Action 2: YDCSearch(query='Moon Jae-In age')
[INFO] Observation 2: Moon Jae-in was born on January 24, 1953.
[INFO] Thought 3: Now I need to calculate Moon Jae-In's age. I'll use the current year to do this.
[INFO] Action 3: GetDatetime(IANA_timezone='Asia/Seoul')
[INFO] Observation 3: timezone: Asia/Seoul, current time(year-month-day-hour-minu

### Reflexion ReAct

In [3]:
import os
from reasoning_engines.langchain_llm_wrappers import QuickAzureChatOpenAI, QuickAzureOpenAIClient
from tools.get_tools import DefaultSchemasTools
from langchain import hub
from utils.load_envs import Environ
from chains.qa_evaluators import CustomQAEvaluator


#### Reflexion Agent
from agents.reflexion_react_agents import ReflexionReActAgent
from chains.reflexion_chains import BaseReflexionChain
from fewshot_examples.reflections import REACT_REFLECTION_HEADER, REACT_REFLECTIONS

GPT_VER = '0613'   
reflexion_chains = BaseReflexionChain(
                    reasoninig_engine=QuickAzureChatOpenAI(version=GPT_VER),
                    prompt="jet-taekyo-lee/experimental-reflextion", 
                    schemas_and_tools=DefaultSchemasTools(azure_gpt_version=GPT_VER),
                    reflexion_examples=REACT_REFLECTIONS
                    )

reflexion_prompt = "jet-taekyo-lee/reflexion-react"
reflexion_agent = ReflexionReActAgent(
                        reflexion_chain = reflexion_chains,
                        reflexion_header = REACT_REFLECTION_HEADER,
                        reasoninig_engine=QuickAzureChatOpenAI(version=GPT_VER), 
                        base_prompt="hwchase17/react-multi-input-json", 
                        schemas_and_tools=DefaultSchemasTools(azure_gpt_version=GPT_VER), 
                        evaluator=CustomQAEvaluator(llm=QuickAzureChatOpenAI(version='1106')), 
                        stop_words=["Observation"], 
                        thought_word= 'Thought', 
                        action_word='Action',
                        file_logging=True,
                        horizon=6)



reflexion_agent.clear_logs()
reflexion_agent.run_agent_trials(num_trials=2, 
query="Who is older? The former president of Korea vs. the MLB American League MVP of the last season. What is the result if you sum up the ages of the two?",
reference="The former former president of Korea is older. The sum of their age is 99.")



[INFO] Query: Who is older? The former president of Korea vs. the MLB American League MVP of the last season. What is the result if you sum up the ages of the two?
[INFO] Trial 1
[INFO] Thought 1: I need to find out who the former president of Korea is and who the MLB American League MVP of the last season is. Then, I need to find out their ages. I'll start by finding out who the former president of Korea is.
[INFO] Action 1: YDCSearch(query='former president of Korea')
[INFO] Observation 1: The most recent former president of South Korea is Moon Jae-In, who served from 2017 to 2022.
[INFO] Thought 2: Now I know that the former president of Korea is Moon Jae-In. Next, I need to find out his age.
[INFO] Action 2: YDCSearch(query='Moon Jae-In age')
[INFO] Observation 2: Moon Jae-in was born on January 24, 1953.
[INFO] Thought 3: Now I know that Moon Jae-In was born on January 24, 1953. I can calculate his age using this information. I'll use the PythonREPL tool to calculate his age.
[INF

### OpenAI FunctionCalling agent

In [4]:
import os
from reasoning_engines.langchain_llm_wrappers import QuickAzureChatOpenAI, QuickAzureOpenAIClient
from tools.get_tools import DefaultSchemasTools
from langchain import hub
from utils.load_envs import Environ
from chains.qa_evaluators import CustomQAEvaluator


# #### OepnAI Agent
GPT_VER = '0613' 
from agents.openai_function_calling_agent import OpenAIFuntionCallingAgent
openai_agent = OpenAIFuntionCallingAgent(
    reasoninig_engine = QuickAzureChatOpenAI(version=GPT_VER),
    base_prompt = "jet-taekyo-lee/openai-function-calling-agent",
    schemas_and_tools=DefaultSchemasTools(azure_gpt_version=GPT_VER),
    evaluator = CustomQAEvaluator(llm=QuickAzureChatOpenAI(version='1106')),
    action_word = 'Invoking',
    horizon = 3)

openai_agent.clear_logs()
openai_agent.run_agent_trials(num_trials=2, 
query="Who is older? The former president of Korea vs. the MLB American League MVP of the last season. What is the result if you sum up the ages of the two?",
reference="The former former president of Korea is older. The sum of their age is 99.")

[INFO] Query: Who is older? The former president of Korea vs. the MLB American League MVP of the last season. What is the result if you sum up the ages of the two?
[INFO] Trial 1
[INFO] Invoking 1: YDCSearch(query='former president of Korea age')
[INFO] Observation 1: The former president of South Korea, Moon Jae-in, was born on January 24, 1953. As of 2022, he is 69 years old. The current president, Yoon Suk Yeol, was born on December 18, 1960, making him 62 years old as of 2022.
[INFO] Invoking 2: YDCSearch(query='MLB American League MVP last season age')
[INFO] Observation 2: The American League MVP for the 2023 season was Shohei Ohtani from the Los Angeles Angels. He was born on July 5, 1994, which would have made him 29 years old during the 2023 season.
[INFO] Invoking 3: PythonREPL(command='print(69 + 29)')
[INFO] Observation 3: 98
[INFO] Jugdement: You failed to provide an answer because you exceeded the permitted number of reasoning steps. You must give an answer within 3 steps

### Reflexion OpenAI FunctionCalling agent

In [5]:
import os
from reasoning_engines.langchain_llm_wrappers import QuickAzureChatOpenAI, QuickAzureOpenAIClient
from tools.get_tools import DefaultSchemasTools
from langchain import hub
from utils.load_envs import Environ
from chains.qa_evaluators import CustomQAEvaluator


# # #### Reflexion OepnAI Agent
from agents.reflexion_openai_function_calling_agent import ReflexionOpenAIFuntionCallingAgent
from chains.reflexion_chains import BaseReflexionChain
from fewshot_examples.reflections import OPENAI_REFLECTIONS, REACT_REFLECTION_HEADER

GPT_VER = '0613'    
openai_reflexion_chains = BaseReflexionChain(
                    reasoninig_engine=QuickAzureChatOpenAI(version=GPT_VER),
                    prompt="jet-taekyo-lee/openai-reflexion-chain-prompt", 
                    schemas_and_tools=DefaultSchemasTools(azure_gpt_version=GPT_VER),
                    reflexion_examples=OPENAI_REFLECTIONS
                    )


openai_reflexion_agent = ReflexionOpenAIFuntionCallingAgent(
                        reflexion_chain = openai_reflexion_chains,
                        reflexion_header = REACT_REFLECTION_HEADER,
                        reasoninig_engine=QuickAzureChatOpenAI(version=GPT_VER), 
                        base_prompt="jet-taekyo-lee/rexlexion-openai-function-calling-agent", 
                        schemas_and_tools=DefaultSchemasTools(azure_gpt_version=GPT_VER),
                        evaluator=CustomQAEvaluator(llm=QuickAzureChatOpenAI(version='1106')),
                        action_word='Invoking',
                        file_logging=True,
                        horizon = 2)

openai_reflexion_agent.clear_logs()
openai_reflexion_agent.run_agent_trials(num_trials=2, 
query="Who is older? The former president of Korea vs. the MLB American League MVP of the last season. What is the result if you sum up the ages of the two?",
reference="The former former president of Korea is older. The sum of their age is 99.")


[INFO] Query: Who is older? The former president of Korea vs. the MLB American League MVP of the last season. What is the result if you sum up the ages of the two?
[INFO] Trial 1
[INFO] Invoking 1: YDCSearch(query='former president of Korea age')
[INFO] Observation 1: The most recent former president of South Korea, Moon Jae-in, was born on January 24, 1953, which makes him 69 years old as of 2022.
[INFO] Invoking 2: YDCSearch(query='MLB American League MVP 2021 age')
[INFO] Observation 2: Shohei Ohtani, who won the American League MVP in 2021, was born on July 5, 1994, which made him 27 years old in 2021.
[INFO] Jugdement: You failed to provide an answer because you exceeded the permitted number of reasoning steps. You must give an answer within 2 steps.
[INFO] Trial 2
[INFO] Reflexion......
[INFO] You have attempted to answer following question before and failed. The following reflection(s) give a plan to avoid failing to answer the question in the same way you did previously. Use th

### Parallel Function Calling agent

In [6]:
import os
from reasoning_engines.langchain_llm_wrappers import QuickAzureChatOpenAI, AsyncQuickAzureOpenAIClient
from tools.get_tools import DefaultSchemasTools
from langchain import hub
from utils.load_envs import Environ
from chains.qa_evaluators import CustomQAEvaluator
import asyncio

#### Parallel Function Calling Agent
GPT_VER = '1106' 
from agents.parallel_func_calling_agent import OpenAIParallelFuntionCallingAgent
parallel_calling_agent = OpenAIParallelFuntionCallingAgent(
    reasoninig_engine = QuickAzureChatOpenAI(version=GPT_VER),
    base_prompt = 'jet-taekyo-lee/parallel-function-calling-agent',
    schemas_and_tools = DefaultSchemasTools(azure_gpt_version=GPT_VER),
    evaluator = CustomQAEvaluator(llm=QuickAzureChatOpenAI('0613')),
    action_word = 'Tool invocations in parallel',
    observation_word = 'Tool-invocations results in parallel',
    use_chat_completion_api = True,
    azure_apenai_client = AsyncQuickAzureOpenAIClient(GPT_VER),
    horizon = 6
)





parallel_calling_agent.clear_logs()
await parallel_calling_agent.run_agent_trials_async(num_trials=2, 
query="Who is older? The former president of Korea vs. the MLB American League MVP of the last season. What is the result if you sum up the ages of the two?",
reference="The former former president of Korea is older. The sum of their age is 99.")



[INFO] Query: Who is older? The former president of Korea vs. the MLB American League MVP of the last season. What is the result if you sum up the ages of the two?
[INFO] Trial 1
[INFO] (step 1-action) Tool invocations in parallel: [Tool 1-> YDCSearch(query=former president of Korea age), Tool 2-> YDCSearch(query=MLB American League MVP last season age)]
[INFO] (step 1-observation) :: [Tool 1-> As of the last update, the most recent former president of South Korea is Moon Jae-in, who was born on January 24, 1953. As of 2023, Moon Jae-in would be 70 years old. If you are asking about a different former president, please specify which one, and I can provide their age or age at the time of their passing., Tool 2-> The American League MVP for the 2023 season was Shohei Ohtani, who was born on July 5, 1994. Since the season typically ends in September or October and the MVP is announced shortly after, Ohtani would have been 29 years old when he was named the AL MVP for the 2023 season.]
[IN

### Reflexion + Parallel Function Calling agent

In [7]:
import os
from reasoning_engines.langchain_llm_wrappers import QuickAzureChatOpenAI, AsyncQuickAzureOpenAIClient
from tools.get_tools import DefaultSchemasTools
from langchain import hub
from utils.load_envs import Environ
from chains.qa_evaluators import CustomQAEvaluator
import asyncio

#### Reflexion + Parallel Function Calling Agent
from chains.reflexion_chains import BaseReflexionChain
from agents.reflexion_parallel_calling_agent import ReflexionOpenAIParallelFuntionCallingAgent
from fewshot_examples.reflections import OPENAI_PARALLEL_CALLING_REFLECTIONS, REACT_REFLECTION_HEADER

GPT_VER = '1106'    
openai_parallel_calling_reflexion_chains = BaseReflexionChain(
                    reasoninig_engine=QuickAzureChatOpenAI(version=GPT_VER),
                    prompt="jet-taekyo-lee/openai-parallel-calling-reflexion-chain-prompt", 
                    schemas_and_tools=DefaultSchemasTools(azure_gpt_version=GPT_VER),
                    reflexion_examples=OPENAI_PARALLEL_CALLING_REFLECTIONS
                    )


openai_parallel_calling_reflexion_agent = ReflexionOpenAIParallelFuntionCallingAgent(
                        reflexion_chain = openai_parallel_calling_reflexion_chains,
                        reflexion_header = REACT_REFLECTION_HEADER,
                        reasoninig_engine=QuickAzureChatOpenAI(version=GPT_VER), 
                        base_prompt="jet-taekyo-lee/reflexion-openai-parallel-calling-agent", 
                        schemas_and_tools=DefaultSchemasTools(azure_gpt_version=GPT_VER),
                        evaluator=CustomQAEvaluator(llm=QuickAzureChatOpenAI(version='1106')),
                        action_word = 'Tool invocations in parallel',
                        observation_word = 'Tool-invocations results in parallel',
                        use_chat_completion_api = True,
                        azure_apenai_client = AsyncQuickAzureOpenAIClient(GPT_VER),
                        file_logging=True,
                        horizon = 6)


openai_parallel_calling_reflexion_agent.clear_logs()
# parallel_calling_agent.run_agent_trials(num_trials=3, 
# query="Which is bigger? The summation of the current Korean president's age and Joe Biden'age vs. the summation of current temperatures of Seoul, Tokyo, and New York?",
# reference="The summation of the current Korean president's age and Joe Biden'age is bigger.")
await openai_parallel_calling_reflexion_agent.run_agent_trials_async(num_trials=2, 
query="Who is older? The former president of Korea vs. the MLB American League MVP of the last season. What is the result if you sum up the ages of the two?",
reference="The former former president of Korea is older. The sum of their age is 99.")



[INFO] Query: Who is older? The former president of Korea vs. the MLB American League MVP of the last season. What is the result if you sum up the ages of the two?
[INFO] Trial 1
[INFO] (step 1-action) Tool invocations in parallel: [Tool 1-> YDCSearch(query=former president of Korea), Tool 2-> YDCSearch(query=MLB American League MVP last season)]


RateLimitError for _get_function_observation. -----> Will automatically retry 20 seconds later.
20 19 18 17 16 15 14 13 12 11 10 9 8 7 6 5 4 3 2 1 RateLimitError for _get_function_observation. -----> Will automatically retry 20 seconds later.
20 19 18 17 16 15 14 13 12 11 10 9 8 7 6 5 4 3 2 1 

[INFO] (step 1-observation) :: [Tool 1-> The most recent former president of South Korea is Moon Jae-in, who served from 2017 to 2022. Prior to Moon Jae-in, the former president was Park Geun-hye, who served from 2013 until her impeachment in 2017., Tool 2-> The American League MVP for the last season (2023) was Shohei Ohtani of the LA Angels.]
[INFO] (step 2-action) Tool invocations in parallel: [Tool 1-> YDCSearch(query=Moon Jae-in age), Tool 2-> YDCSearch(query=Shohei Ohtani age)]
[INFO] (step 2-observation) :: [Tool 1-> Moon Jae-in was born on January 24, 1953. As of the time of your question, he is 70 years old., Tool 2-> Shohei Ohtani is 29 years old.]
[INFO] (step 3-action) Tool invocations in parallel: Now I can answer directly without resorting to any tool.
[INFO] Answer: Shohei Ohtani is the younger of the two. The sum of their ages is 99 years.
[INFO] Jugdement: Your answer is correct.
